# Getting Started

This package allow the user to define lattice geometries easily using a nested structure of orbitals, sites and unit cells. Once this lattice geometry is constructed, the user can define in a natural manner the coupling terms between each sites, fix the boundary conditions, and let the HamiltonianBuilder class construct the Hamiltonian.

This package was designed to construct large, multidimensional Hamiltonian arrays, dependant on multiple parameters, and particularly reciprocal space vectors in the case of Bloch Hamiltonian. Under the hood, it uses <xarray.DataArray> structures to create structured and lisible arrays.

## Constructing the lattice

First, the lattice is built as a nested structure of various objects. Let's import these classes.

In [1]:
from tightbinding.geometry import Orbital, Site, Unitcell, Lattice

A Lattice class object is made of one (or many) Unitcell class object, each made of Site objects, each made of Orbital objects. This structure and the periodic nature of a lattice ensures that no extra effort than neccessary is needed to build the lattice geometry.

Let's first look at the Orbital class.

In [2]:
s_orbital = Orbital("s", radius=1)
print(s_orbital)

Orbital s: radius = 1, polarization = 0, angle dependance = (0.0,)


A orbital object is a very simple object, basically containing only a name and a few attributes. These attributes are as for now not used in the package, but further plotting and computing updates. Any number of orbital objects can be created, then used to create a Site object. Let's create one more orbital, then create a site:

In [3]:
px_orbital = Orbital("px")

A_site = Site("A", [-1,0], [s_orbital, px_orbital])
print(A_site)

Site A: 
 Position = [-1  0]
 Orbitals: ('s', 'px')


We have created a Site Object, named A, containing two orbitals and placed at the position $[-1,0]$. This position is the one that the site will occupy in the unit cell, respective to its center. The position list must have as many arguments as the dimensionality of your lattice. The module can handle 1-D, 2-D, 3-D or even N-D lattices. A Site object contains a 'orbitals' dictionnary, in which all orbital objects contained can be accessed through their name. 

Let's contruct now a second site, and use it to assemble our first unit cell.

In [4]:
B_site = Site("B", [1,0], s_orbital)
B_site.add_orbital(px_orbital) # It is possible to add orbitals to a site after initialization if needed

A_site.remove_orbital("px") # It is also possible to remove orbitals from a site.
B_site.remove_orbital("px") # It is also possible to remove orbitals from a site.

a1 = [3, -3**0.5]
a2 = [3, 3**0.5]

uc = Unitcell("honeycomb", [a1,a2])
uc.add_site([A_site, B_site])
print(uc)

honeycomb (sites ('A', 'B'))


We have now created our unit cell, which the astute reader might recognize as the honeycomb lattice unit cell. This unit cell is first defined by a list of lattice vectors, with as many elements as the dimensionality of the lattice. Like a Site object, a Unitcell object contains a 'sites' dictionary of its contained sites, where each site can be accessed through its name. We can also plot a unit cell using the .plot() function:

In [5]:
fig = uc.plot()

The 'plot' function displays the unit cell with each of its sites, and lists the orbitals withing each sites, allowing for a quick visual check of the constructed structure. It can show either 2D or 3D unit cells, using the plotly library for interactivity.

We are finally ready to construct our lattice object. Lattice, here, is defined in a broad sense, as a Lattice object can be composed of a single unit cell with periodic boundary conditions, in this case, the resulting Hamiltonian is simply the lattice's Bloch Hamiltonian.

In [6]:
honeycomb = Lattice("honeycomb", unitcell=uc, periodicity=(False, False))
print(honeycomb)

A Lattice object with unit cell honeycomb


We now have created an empty lattice, with a single unit cell template object attached. In the future, creation of super lattices made of different unit cell might be supported. For now, this lattice object is empty, and we will have to add the unit cell ourselves. We also have set the periodicity of the lattice to 'False', in both dimensions. This periodicity is defined along the lattice vectors, here provided by the only unit cell object. Now, let's add some unit cell and plot the lattice:

In [7]:
honeycomb.add_unitcell((0,0))
honeycomb.add_unitcell((1,0))
honeycomb.add_unitcell((0,1))
honeycomb.add_unitcell((0,-1))
honeycomb.add_unitcell((2,2), update=True)

honeycomb.plot()

We have added a few unit cells by hand, using the low-level 'add_unitcell' function. The position of the unit cell was specified in the lattice vector coordinates unit. In the last call, we also fixed the 'update' argument to 'True', which tells the function to update the lattice index, a dictionnary containing all orbitals of each sites of each unit cell. This update is usually performed automatically by the high-level builder function, such as this one:

In [8]:
honeycomb.create_rectangle_lattice((6,6))
honeycomb.plot()

We now have a larger lattice, let's clean it up by removing this extra unit cell sticking out.

In [9]:
honeycomb.remove_unit_cell((0,-1))

We now have a simple honeycomb lattice, made of $5 \times 5$ unit cells. We are now ready to go to the next step.

## Building the Hamiltonian

A tight-binding Hamiltonian can be written in this basis form :

$$\hat{H} = \sum_{i}\epsilon_i \hat{a}^\dagger_i \hat{a}_i + \sum_{i,j} t_{ij} \hat{a}^\dagger_j \hat{a}_i +c.c.$$

With $\epsilon_i$ the on-site energy of the i-th orbital, $t_{ij}$ the coupling term between orbitals i and j and $\hat{a}_i$ ($\hat{a}^\dagger_i$) the destruction (creation) operator of orbital i. This Hamitlonian can be represented by a $N \times N$ Hermitian matrix in the basis of these operators, with N the total number of orbitals in the lattice, and the submodule 'hamiltonian' contains three classes that allow the user to easily build this matrix: ''Energy', 'Hopping' and 'HamiltonianBuilder'.

### Hamiltonian parameters

Before contructing the Hamitlonian proper, we have to define its parameters, some might be fixed, some we might want to explore.

In [10]:
import numpy as np

t = -1 # the hopping coefficient
epsilon0 = np.linspace(-1, 1, 21) # The on-site energy difference between sites A and B
strain = np.linspace(0, 2, 21)

parameters = {
    "t": t,
    "eps0": epsilon0,
    "strain": strain,
    }

Here, we have three parameters for our Hamiltonian, the hopping amplitude $t$, the on-site energy difference $\epsilon_0$ and the strain of the lattice in the x-axis. We can transmit these informations to the code by instancing Hopping objects and a Energy object

### Hopping and Energy class objects

In [11]:
from tightbinding.hamiltonian import Energy, Hopping, Hamiltonianbuilder

onsite = Energy(honeycomb)
onsite.set_energy("-eps0/2", nsite='A')
onsite.set_energy("eps0/2", nsite='B')

coupling1 = Hopping(honeycomb, "strain * t", "A_s", "B_s", displacement=(0,0))
coupling2 = Hopping(honeycomb, "t", "A_s", "B_s", displacement=(-1,0))
coupling3 = Hopping(honeycomb, "t", "A_s", "B_s", displacement=(0,-1))

First, we have created an 'Energy' object. The Energy class is a small class that allows you to define the energy of each orbital in each sites of the lattice. Using the function set_energy, you can give the expression of the energy term, as a function of the parameters previoulsy defined in the dictionary. three keyword arguments 'norbital', 'nsite' and 'nunitcell' allow you to specify which orbitals to affect. The function affects all orbitals that matches the names keyword given. For exemple, here, we have specified only the site names, which means that each orbital of each unit cell in the proper site is affected. Another function, add_energy, allows to add additional terms to the energy expression. It is also possible to fix a boolean condition, taht can depend on some unit cell index or position to determine wheter an orbital is affected by the setting of the energy term (see examples latter).

We then have created three coupling terms, using the Hopping class. These objects require an expression, which gives the amplitude of the coupling term, a starting orbital (in the unit cell), given as '<site name>_<orbital name>', and a displacement describing the number of unit cell between the two orbitals. Here, we only consider the nearest-neighbour hopping between the A and B sites. We are now ready to construct the Hamiltonian.

### The Hamiltonian builder class

In [12]:
Hamiltonian = Hamiltonianbuilder(honeycomb, parameters, reciprocalcoords=[])

The HamiltonianBuilder class constructor takes as argument a lattice, a dictionary of parameters that can be either constants, numpy arrays or xarray DataArrays and a list of reciprocal coordinates. This last argument is useful if the boundaries are periodic (see example). Its main element is the 'Hamiltonian' xr.DataArray. A wrapper around a numpy array that supports label-based operation (see [xarray doc](https://docs.xarray.dev/en/stable/index.html)). Right now, the Hamiltonian matrix is empty, let's fill it.

In [13]:
Hamiltonian.set_on_site_energies(onsite)
Hamiltonian.add_couplings([coupling1,coupling2,coupling3])
Hamiltonian.build()

Here, we added the Energy object 'onsite' and Hopping objects to the builder, then called the 'build' function, which fills the array, let's see how it looks:

In [14]:
print(Hamiltonian.Hamiltonian)

<xarray.DataArray 'Hamiltonian' (eps0: 21, strain: 21, i: 72, j: 72)> Size: 37MB
array([[[[ 0.5 +0.j,  0.  +0.j,  0.  +0.j, ...,  0.  +0.j,  0.  +0.j,
           0.  +0.j],
         [ 0.  +0.j, -0.5 +0.j, -1.  +0.j, ...,  0.  +0.j,  0.  +0.j,
           0.  +0.j],
         [ 0.  +0.j, -1.  +0.j,  0.5 +0.j, ...,  0.  +0.j,  0.  +0.j,
           0.  +0.j],
         ...,
         [ 0.  +0.j,  0.  +0.j,  0.  +0.j, ..., -0.5 +0.j, -1.  +0.j,
           0.  +0.j],
         [ 0.  +0.j,  0.  +0.j,  0.  +0.j, ..., -1.  +0.j,  0.5 +0.j,
           0.  +0.j],
         [ 0.  +0.j,  0.  +0.j,  0.  +0.j, ...,  0.  +0.j,  0.  +0.j,
          -0.5 +0.j]],

        [[ 0.5 +0.j, -0.1 +0.j,  0.  +0.j, ...,  0.  +0.j,  0.  +0.j,
           0.  +0.j],
         [-0.1 +0.j, -0.5 +0.j, -1.  +0.j, ...,  0.  +0.j,  0.  +0.j,
           0.  +0.j],
         [ 0.  +0.j, -1.  +0.j,  0.5 +0.j, ...,  0.  +0.j,  0.  +0.j,
           0.  +0.j],
...
         [ 0.  +0.j,  0.  +0.j,  0.  +0.j, ...,  0.5 +0.j, -1.  +0.j,
 

We can see that this array is 4-dimensional. The first two dimensions are named "eps0" and "strain", they represent the two parameter array we defined earlier. The last two dimensions are always called "i" and "j" and are the two dimensions of each Hamiltonian matrix. 

It is possible to get a visual representation of all the coupling terms in Hamiltonian, by first building it then calling the function "plot_coupling"

In [15]:
Hamiltonian.plot_coupling()

    'data': [{'hoverinfo': 'text',
              'line': {'color': 'rgba(100,100…

This function is a plotly interactive figure, which might not be compatible with all IDEs, it is mainly made to work with jupyter notebooks. It shows the coupling terms between each lattice site as a line with additional information on hover. Sliders for each of the parameter dimensions allow to track in real time the coupling strength and verify thta the proper terms where included.

## Data analysis

Once the Hamiltonian has been constructed, the first natural step is of course to diagonalize it. This can be done easily by the eigh or eigh_parallel methods of the module. eigh_parallel is a [dask](https://www.dask.org/) powered parralel diagonalization, it is not recommended for a small number of matrices, but can be very useful for larger arrays.

In [16]:
eigva, vecs = Hamiltonian.eigh()

from tightbinding.plotting import plot_density
plot_density(eigva)

Here, we first imported the plot_density function from the plotting submodule, and used it to generate an interactive plot of the density of states. More functions exists in this submodules, and are all presented in the various example notebooks. Finally, we will conclude here by looking at the spatial profile of the eigenvectors, we can use the plot_field function of the Lattice class.

In [17]:
honeycomb.plot_field(abs(vecs), orbital='s', vtype='amplitude')

    'data': [{'hoverinfo': 'all',
              'marker': {'color': {'bdata': ('…